In [9]:
import pickle
from IPython.display import Video
import importlib
import numpy as np
%cd stac-mjx
import viz
import os
# os.environ["IMAGEIO_FFMPEG_EXE"] = "/usr/bin/ffmpeg"
# viz = importlib.import_module("stac-mjx.viz")

param_path = "../params/params.yaml"
offset_path = "../offset.p"
n_frames = 500

[Errno 2] No such file or directory: 'stac-mjx'
/Users/charleszhang/GitHub/stac-mjx/stac-mjx


In [10]:
save_path = "../videos/mujoco_video.mp4"
viz.render_mujoco(
    param_path,
    offset_path,
    save_path,
    frames = np.arange(n_frames),
)
Video(save_path)

(500, 74)
walker: <walkers.Rat object at 0x3f65a3290>
arena: <arenas.DannceArena object at 0x4132b0110>


### rendering directly via mujoco

In [1]:
import mediapy as media
from dm_control import mjcf
from dm_control.mujoco.wrapper.mjbindings import enums
import mujoco
from jax import numpy as jnp
from mujoco import mjx
import pickle
import imageio
import numpy as np
import os
%cd stac-mjx
import controller
import utils
import stac_base

os.environ["MUJOCO_GL"] = "osmesa"
offset_path = "../LM_fit.p"

scene_option = mujoco.MjvOption()
# scene_option.geomgroup[1] = 0
scene_option.geomgroup[2] = 1
# scene_option.geomgroup[3] = 0
# scene_option.sitegroup[0] = 0
# scene_option.sitegroup[1] = 0
scene_option.sitegroup[2] = 1

scene_option.sitegroup[3] = 1
scene_option.flags[enums.mjtVisFlag.mjVIS_TRANSPARENT] = True
scene_option.flags[enums.mjtVisFlag.mjVIS_LIGHT] = False
scene_option.flags[enums.mjtVisFlag.mjVIS_CONVEXHULL] = True
scene_option.flags[enums.mjtRndFlag.mjRND_SHADOW] = False
scene_option.flags[enums.mjtRndFlag.mjRND_REFLECTION] = False
scene_option.flags[enums.mjtRndFlag.mjRND_SKYBOX] = False
scene_option.flags[enums.mjtRndFlag.mjRND_FOG] = False

/home/charles/github/stac-mjx/stac-mjx


/home/charles/miniconda3/envs/stac-mjx/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
utils.init_params("../params/params.yaml")

# load mjx_model and mjx_data and set marker sites
rat_xml = "../models/rodent_stac.xml"
root = mjcf.from_path(rat_xml)
physics, mj_model = controller.create_body_sites(root)
physics, mj_model, keypoint_sites = controller.create_keypoint_sites(root)

#starting xpos anda xquat for mjdata
_UPRIGHT_POS = (0.0, 0.0, 0.94)
_UPRIGHT_QUAT = (0.859, 1.0, 1.0, 0.859)

In [3]:
# mjx_data = stac_base.kinematics(mjx_model, mjx_data)
mj_data = mujoco.MjData(mj_model)
mj_data.xpos = _UPRIGHT_POS
mj_data.xquat = _UPRIGHT_QUAT
mujoco.mj_kinematics(mj_model, mj_data)

### Full video rendering directly using mujoco

In [4]:
# open data file, loop through qpos blah blah
# offset_path="../LM_fit.p"
with open(offset_path, "rb") as file:
    d = pickle.load(file)
    qposes = np.array(d["qpos"])
    kp_data = np.array(d["kp_data"])


FPS=50
n_frames = 1000
frames=[]
save_path="../videos/direct_render.mp4"
renderer = mujoco.Renderer(mj_model, height=1200, width=1920)
# Make sure there are enough frames to render
if qposes.shape[0] < n_frames-1:
    raise Exception(f"Trying to render {n_frames} frames when data['qpos'] only has {qposes.shape[0]}")

# slice kp_data to match qposes length
kp_data = kp_data[:qposes.shape[0]]

# render while stepping using mujoco, not mjx
with imageio.get_writer(save_path, fps=FPS) as video:
    for i, (qpos, kps) in enumerate(zip(qposes, kp_data)):
        if i%100 == 0:
            print(f"rendering frame {i}")
        if i == n_frames:
            break
        
        #set keypoints
        physics, mj_model = controller.set_keypoint_sites(physics, keypoint_sites, kps)
        # print(mj_data.qpos)
        # print(qpos)
        mj_data.qpos = qpos
        # print(mj_data.qpos)
        mujoco.mj_forward(mj_model, mj_data)

        # mjx_model = mjx.put_model(mj_model)
        # 
        # print(f"the deltas: {(stac_base.get_site_xpos(mjx_data).flatten() - kps)}")
        # set mjx_data.qpos
        # mjx_data = mjx_data.replace(qpos=jnp.array(qpos))
        # mjx_data = stac_base.kinematics(mjx_model, mjx_data)
        # mjx_data = stac_base.com_pos(mjx_model, mjx_data)

        # render
        
        # mj_data = mjx.get_data(mj_model, mjx_data)
        renderer.update_scene(mj_data, camera="close_profile", scene_option=scene_option)
        pixels = renderer.render()
        video.append_data(pixels)
        frames.append(pixels)

media.show_video(frames, fps=FPS)

/home/charles/miniconda3/envs/stac-mjx/lib/python3.12/site-packages/imageio_ffmpeg/_utils.py:7: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_filename


rendering frame 0
rendering frame 100
rendering frame 200
rendering frame 300
rendering frame 400
rendering frame 500
rendering frame 600
rendering frame 700
rendering frame 800
rendering frame 900


This browser does not support the video tag.


In [ ]:
import stac_base

markers = stac_base.get_site_xpos(mjx_data).flatten() 
markers

Array([ 3.12635173e-01, -2.14071056e-03, -2.48040521e-03,  3.51357973e-01,
       -2.22390872e-02, -3.40437055e-03,  3.86493085e-01,  8.84612340e-02,
        9.14292293e-02,  3.86493085e-01,  8.84612340e-02,  9.14292293e-02,
        3.54778418e-01,  7.22992018e-02,  4.19854925e-02,  3.89655146e-01,
        5.41970604e-02,  4.11532984e-02,  3.15315077e-01,  1.77498472e-02,
        8.21022999e-04,  3.66199973e-01, -8.66104732e-03, -3.93142284e-04,
        3.56336734e-01,  1.03692915e-01,  2.44815935e-02,  4.13762718e-01,
        7.38869851e-02,  2.31113512e-02,  3.16679625e-01, -1.47644321e-02,
        4.87246209e-02,  3.40638594e-01, -2.71999059e-02,  4.81529355e-02,
        3.19526877e-01,  1.91083317e-02,  2.50585532e-02,  3.65800034e-01,
       -4.90892253e-03,  2.39544286e-02,  3.67115802e-01,  7.47686623e-02,
        6.51602119e-02,  3.85452400e-01,  6.52513796e-02,  6.47226821e-02,
        3.86493085e-01,  8.84612340e-02,  9.14292293e-02,  3.65866570e-01,
        4.97214487e-02,  

In [36]:
kps = data["kp_data"][0]

In [37]:
jnp.sum(jnp.square((kps - markers))) * 0.5

Array(0.00552002, dtype=float64)

In [38]:
jnp.abs(kps - markers)

Array([0.00913613, 0.01027233, 0.0158153 , 0.00487711, 0.00987631,
       0.01607085, 0.02204112, 0.00893821, 0.01925653, 0.00338292,
       0.00760634, 0.01167895, 0.00932603, 0.0014955 , 0.00393766,
       0.00415304, 0.00130805, 0.00473414, 0.01510866, 0.01028148,
       0.00833713, 0.00058933, 0.00640033, 0.00911062, 0.00640427,
       0.01681358, 0.01538146, 0.02596203, 0.00613267, 0.01428621,
       0.0006225 , 0.01716347, 0.0004286 , 0.00539137, 0.01105097,
       0.00054501, 0.00351097, 0.00871465, 0.01171861, 0.00117845,
       0.00400179, 0.00961844, 0.01960247, 0.00800546, 0.0008335 ,
       0.00742081, 0.01408536, 0.00039182, 0.02929329, 0.02612497,
       0.0252084 , 0.0041715 , 0.00726707, 0.01161732, 0.01064545,
       0.00734256, 0.00497375, 0.00731071, 0.00847428, 0.00788616,
       0.01912909, 0.0136389 , 0.0296868 , 0.00690708, 0.00672485,
       0.02139078, 0.00467721, 0.01570162, 0.02077338], dtype=float64)

In [39]:
jnp.square(kps - markers)

Array([8.34688342e-05, 1.05520803e-04, 2.50123604e-04, 2.37861818e-05,
       9.75414882e-05, 2.58272301e-04, 4.85810937e-04, 7.98916078e-05,
       3.70813962e-04, 1.14441648e-05, 5.78563700e-05, 1.36397804e-04,
       8.69747564e-05, 2.23651585e-06, 1.55051967e-05, 1.72477426e-05,
       1.71099819e-06, 2.24120403e-05, 2.28271585e-04, 1.05708774e-04,
       6.95076820e-05, 3.47314664e-07, 4.09642781e-05, 8.30034852e-05,
       4.10146660e-05, 2.82696521e-04, 2.36589240e-04, 6.74026939e-04,
       3.76096196e-05, 2.04095720e-04, 3.87501978e-07, 2.94584707e-04,
       1.83701536e-07, 2.90669181e-05, 1.22123909e-04, 2.97034915e-07,
       1.23268771e-05, 7.59451431e-05, 1.37325851e-04, 1.38874799e-06,
       1.60143263e-05, 9.25143090e-05, 3.84256681e-04, 6.40873230e-05,
       6.94724637e-07, 5.50684777e-05, 1.98397372e-04, 1.53522785e-07,
       8.58096980e-04, 6.82514083e-04, 6.35463255e-04, 1.74014290e-05,
       5.28102924e-05, 1.34962236e-04, 1.13325605e-04, 5.39131624e-05,
      

### rendering single frame via dmcontrol

In [3]:
%cd stac-mjx
import numpy as np

import viz
from IPython.display import Video
import matplotlib.pyplot as plt

offset_path = "root.p"
import pickle
with open(offset_path, "rb") as file:
    data = pickle.load(file)

data["qpos"] = np.array(data["qpos"])
data["xpos"] = np.array(data["xpos"])
# data["walker_body_sites"] = np.array(data["walker_body_sites"])

data["qpos"] = np.tile(data["qpos"], (3, 1))
data["xpos"] = np.tile(data["xpos"], (3, 1, 1))
# data["walker_body_sites"] = np.tile(data["walker_body_sites"], (3, 1, 1))

with open(offset_path, "wb") as output_file:
    pickle.dump(data, output_file, protocol=2)
save_path = "../videos/root.mp4"
frame = viz.render_first(
    "../params/params.yaml",
    offset_path,
    save_path,
    frames = np.arange(3),
)

/Users/charleszhang/miniforge3/envs/stac-mjx/lib/python3.12/site-packages/IPython/core/magics/osm.py:393: UserWarning: using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


[Errno 2] No such file or directory: 'stac-mjx'
/Users/charleszhang/GitHub/stac-mjx/stac-mjx
(6561, 74)
walker: <walkers.Rat object at 0x16b0b5a60>
arena: <arenas.DannceArena object at 0x2a669b920>
2.0
